### 原理
#### 核心机制
1. **哈希表存储**：使用字典存储函数参数和返回值的映射关系  
2. **参数作为键**：将函数的所有参数转换为可哈希的键  
3. **结果缓存**：相同的参数调用直接返回缓存结果，避免重复计算

底层实现

In [ ]:
# 简化的cache实现原理
def cache(user_function):
    cache_dict = {}
    
    def wrapper(*args, **kwargs):
        # 创建缓存键
        key = (args, tuple(kwargs.items()))
        
        if key not in cache_dict:
            # 缓存未命中，执行函数并缓存结果
            cache_dict[key] = user_function(*args, **kwargs)
        
        # 返回缓存结果
        return cache_dict[key]
    
    return wrapper

#### 使用方式
基本使用

In [ ]:
from functools import cache

@cache
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

# 第一次计算会执行函数
print(fibonacci(10))  # 55
# 第二次相同参数直接返回缓存结果
print(fibonacci(10))  # 立即返回55

带参数的函数

In [ ]:
@cache
def expensive_computation(a, b, c=10):
    print(f"计算 {a}, {b}, {c}")  # 只会打印一次
    return a * b * c

result1 = expensive_computation(3, 4, c=5)  # 会打印
result2 = expensive_computation(3, 4, c=5)  # 不会打印，使用缓存

与 `@lru_cache` 的区别
|特性|	@cache	|@lru_cache|
|:-:|:-:|:-:|
|缓存大小|	无限制|	可限制大小|
|内存管理	|可能内存泄漏|	LRU淘汰策略|
|Python版本|	3.9+|	3.2+|

In [ ]:
from functools import lru_cache

# @cache 等价于 @lru_cache(maxsize=None)
@cache  # 无限制缓存
def func1():
    pass

@lru_cache(maxsize=None)  # 同上
def func2():
    pass

@lru_cache(maxsize=128)  # 限制缓存大小
def func3():
    pass

#### 实际应用场景
1. 递归优化


In [ ]:
@cache
def combinations(n, k):
    if k == 0 or k == n:
        return 1
    return combinations(n-1, k-1) + combinations(n-1, k)

print(combinations(50, 25))  # 大量重复计算被优化

2. API调用缓存

In [ ]:
import requests
from functools import cache
import time

@cache
def get_weather(city):
    print(f"调用API获取{city}天气")
    # 模拟API调用
    time.sleep(2)
    return f"{city}: 晴朗"

# 第一次调用实际请求
weather1 = get_weather("北京")  # 打印并等待
# 后续相同参数使用缓存
weather2 = get_weather("北京")  # 立即返回

3. 配置加载

In [ ]:
@cache
def load_config(config_path):
    print(f"加载配置文件: {config_path}")
    # 模拟读取文件
    with open(config_path, 'r') as f:
        return f.read()

# 多次调用只加载一次
config1 = load_config("config.json")
config2 = load_config("config.json")  # 使用缓存

#### 注意事项  
1. 参数必须可哈希

In [ ]:
@cache
def process_data(data):
    return sum(data)

# 错误：列表不可哈希
# process_data([1, 2, 3])

# 正确：使用元组
process_data((1, 2, 3))

2. 副作用函数慎用

In [ ]:
counter = 0

@cache
def with_side_effect():
    global counter
    counter += 1
    return counter

print(with_side_effect())  # 1
print(with_side_effect())  # 1（不是2！）

3. 内存使用监控

In [ ]:
import sys
from functools import cache

@cache
def large_memory_function(n):
    return [i for i in range(n)]

# 缓存大量结果可能导致内存问题
results = []
for i in range(1000):
    results.append(large_memory_function(10000))

#### 高级技巧  
1. 手动缓存控制

In [ ]:
from functools import cache

@cache
def cached_func(x):
    return x * x

# 查看缓存信息
print(cached_func.cache_info())

# 清空缓存
cached_func.cache_clear()

2. 类方法缓存

In [ ]:
from functools import cache

class DataProcessor:
    @staticmethod
    @cache
    def process(data_key):
        # 复杂的处理逻辑
        return f"processed_{data_key}"
    
    @classmethod
    @cache
    def class_process(cls, data_key):
        return f"{cls.__name__}_processed_{data_key}"

3. 条件缓存

In [ ]:
from functools import cache

def conditional_cache(condition=True):
    def decorator(func):
        if condition:
            return cache(func)
        return func
    return decorator

@conditional_cache(condition=True)
def cached_when_needed(x):
    return expensive_operation(x)

### 总结
`@cache` 是一个简单但强大的性能优化工具，特别适用于：  
+ 纯函数（相同输入总是相同输出）  
+ 计算密集型函数
+ 递归函数优化
+ 重复调用相同参数的函数

使用时应考虑：  
+ 函数参数是否可哈希
+ 缓存是否会导致内存问题
+ 函数是否有副作用

对于需要限制内存使用的场景，建议使用 `@lru_cache(maxsize=...)` 替代。